In [28]:
import json
from pathlib import Path
import os
import pickle
import pandas as pd

path="C:/Users/chris/DSC650-T301/dsc650/dsc650/assignments/assignment02"

os.chdir(path)

In [29]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)



#people_pickle = kv_data_dir.joinpath('people.pkl')
#visited_pickle = kv_data_dir.joinpath('visited.pkl')
#sites_pickle = kv_data_dir.joinpath('sites.pkl')
#measurements__pickle = kv_data_dir.joinpath('measurements.pkl')

people_json = kv_data_dir.joinpath('people.json')
visited_json = kv_data_dir.joinpath('visited.json')
sites_json = kv_data_dir.joinpath('sites.json')
measurements_json = kv_data_dir.joinpath('measurements.json')




In [30]:
class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            #with open(self._db_path, 'rb') as f:
            #    self._db = json.load(f)
            with open(self._db_path, 'a+') as f:
                json.dump(self._db, f)
   
            #with open(self._db_path, 'rb') as file:
            #    print(self._db_path)
            #    response = pickle.load(file)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
        #with open(self._db_path, 'wb') as f:
            #pickle.dump(self._db, f, protocol=pickle.HIGHEST_PROTOCOL)
            json.dump(self._db, f, indent=2, skipkeys=True)
     

In [31]:
def create_sites_kvdb():
    db = KVDB(sites_json)
    df = pd.read_csv('C:/Users/chris/DSC650-T301/dsc650/data/external/tidynomicon/site.csv')
    for site_id, group_df in df.groupby('site_id'):
        db.set_value(site_id, group_df.to_dict(orient='records')[0])
        print(db.get_value(site_id))
    db.save()
    
def create_people_kvdb():
    db = KVDB(people_json)
    df = pd.read_csv('C:/Users/chris/DSC650-T301/dsc650/data/external/tidynomicon/person.csv')
    for person_id, group_df in df.groupby('person_id'):        
        db.set_value(person_id, group_df.to_dict(orient='records')[0])
        print(db.get_value(person_id))
    db.save()

def create_visits_kvdb():
    db = KVDB(visited_json)
    df = pd.read_csv('C:/Users/chris/DSC650-T301/dsc650/data/external/tidynomicon/visited.csv')
    for visit_id, group_df in df.groupby(["visit_id","site_id"]):
        visit_id_str= str(visit_id)
        db.set_value(visit_id_str, group_df.to_dict(orient='records')[0])
        print(db.get_value(visit_id_str))
    db.save()

def create_measurements_kvdb():
    db = KVDB(measurements_json)
    df = pd.read_csv('C:/Users/chris/DSC650-T301/dsc650/data/external/tidynomicon/measurements.csv')
    for person_id, group_df in df.groupby(["visit_id","person_id","quantity"]):
        person_id_str = str(person_id)
        db.set_value(person_id_str, group_df.to_dict(orient='records')[0])
        print(db.get_value(person_id_str))
    db.save()



In [32]:
create_sites_kvdb()
create_people_kvdb()
create_measurements_kvdb()
create_visits_kvdb()

{'site_id': 'DR-1', 'latitude': -49.85, 'longitude': -128.57}
{'site_id': 'DR-3', 'latitude': -47.15, 'longitude': -126.72}
{'site_id': 'MSK-4', 'latitude': -48.87, 'longitude': -123.4}
{'person_id': 'danforth', 'personal_name': 'Frank', 'family_name': 'Danforth'}
{'person_id': 'dyer', 'personal_name': 'William', 'family_name': 'Dyer'}
{'person_id': 'lake', 'personal_name': 'Anderson', 'family_name': 'Lake'}
{'person_id': 'pb', 'personal_name': 'Frank', 'family_name': 'Pabodie'}
{'person_id': 'roe', 'personal_name': 'Valentina', 'family_name': 'Roerich'}
{'visit_id': 619, 'person_id': 'dyer', 'quantity': 'rad', 'reading': 9.82}
{'visit_id': 619, 'person_id': 'dyer', 'quantity': 'sal', 'reading': 0.13}
{'visit_id': 622, 'person_id': 'dyer', 'quantity': 'rad', 'reading': 7.8}
{'visit_id': 622, 'person_id': 'dyer', 'quantity': 'sal', 'reading': 0.09}
{'visit_id': 734, 'person_id': 'lake', 'quantity': 'sal', 'reading': 0.05}
{'visit_id': 734, 'person_id': 'pb', 'quantity': 'rad', 'reading'

In [33]:
kvdb_path = 'visited.json'
kvdb = KVDB(kvdb_path)
key = (619, 'DR-1')
value = dict(
    visit_id=619,
    site_id='DR-1',
    visit_date='1927-02-08'
)
kvdb.set_value(key, value)
retrieved_value = kvdb.get_value(key)
retrieved_value

{'visit_id': 619, 'site_id': 'DR-1', 'visit_date': '1927-02-08'}